# 1. Explore on Data Set

In [44]:
import pymongo
import pprint


In [45]:
db = pymongo.MongoClient(host=["75.80.50.2:27017"], \
                         username="team", password="team2020", authSource="utility_patents_full")

In [46]:
db


MongoClient(host=['75.80.50.2:27017'], document_class=dict, tz_aware=False, connect=True, authsource='utility_patents_full')

In [47]:
db.utility_patents_full

Database(MongoClient(host=['75.80.50.2:27017'], document_class=dict, tz_aware=False, connect=True, authsource='utility_patents_full'), 'utility_patents_full')

In [48]:
db.utility_patents_full.comprehensive.list_indexes()


In [49]:
for index in db.utility_patents_full.comprehensive.list_indexes():
    print(index)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'utility_patents_full.comprehensive')])
SON([('v', 2), ('unique', True), ('key', SON([('applicationNumber', 1)])), ('name', 'applicationNumber'), ('ns', 'utility_patents_full.comprehensive')])
SON([('v', 2), ('key', SON([('publications.publicationNumber', 1)])), ('name', 'publicationNumberNonUnique'), ('ns', 'utility_patents_full.comprehensive'), ('background', True)])
SON([('v', 2), ('unique', True), ('key', SON([('applicationNumber', 1), ('publications.publicationNumber', 1)])), ('name', 'application_publication'), ('ns', 'utility_patents_full.comprehensive'), ('sparse', False), ('background', True)])
SON([('v', 2), ('key', SON([('applicationStatusCategory', -1)])), ('name', 'applicationStatus'), ('ns', 'utility_patents_full.comprehensive'), ('background', True)])
SON([('v', 2), ('key', SON([('filingDate', -1)])), ('name', 'filingDate'), ('ns', 'utility_patents_full.comprehensive'), ('background', True)])
SON([('v', 

In [50]:
db.list_database_names()

['utility_patents_full']

In [51]:
patents = db['utility_patents_full']

In [52]:
comprehensive = patents['comprehensive']
comprehensive.find_one()['publications'][0]['fullText'].keys()


dict_keys(['inventionTitle', 'abstract', 'summaryOfInvention', 'detailedDescription', 'briefDescriptionOfDrawings', 'claims'])

In [53]:
comprehensive.find_one()#['publications'][0]['fullText']['claims']

{'_id': ObjectId('5e736b26aefb9df49c5e5dcf'),
 'applicationNumber': '12923265',
 'publications': [{'publicationNumber': '20110061141',
   'kind': 'A1',
   'fullText': {'inventionTitle': 'Protective composite fabric',
    'abstract': ['The invention relates to a slash and gash resistant protective composite fabric comprising at least one layer of base fabric, wherein the base fabric is a densely woven, ballistic fabric. At least one layer of rigid fabric, each having a base layer and a hard polymer layer having a special matrix, is bonded onto the base layer, and the rigid fabric is cut into strips of material having a scale-like pattern or shape. Each of the strips of material are connected to the layer of base fabric in overlapping layers whereby the base fabric is covered. The protective composite fabric further includes a resilient elastically deformable material having at least one sensor therein for monitoring of vital signs of a wearer, the resilient elastically deformable materi

In [54]:
comprehensive.find_one()['simpleLabelFromStatusByFinalRejection']

0

In [55]:
# comprehensive.count_documents({})



# 2. Extract Field on DB

In [56]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import re
import string


In [57]:
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yifeining/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
def clean_text(claim):
    claim = claim.lower()
    for tag in ['DETAILED DESCRIPTION OF THE PREFERRED EMBODIMENT', 'BACKGROUND OF THE INVENTION']:
        claim = claim.replace(tag, '')
    words = [s for s in word_tokenize(claim) if s not in string.punctuation]
    words = [w for w in words if not w in stop_words]
    words = [s for s in words if not re.match(r'\d', s)]
    return words


def get_claim(obj):
    try:
        concat_claims = ' '.join([clm['claim'] for clm in obj['publications'][0]['fullText']['claims']])
    except:
        return None
    out = clean_text(concat_claims)
    return out


def get_label(obj):
    try:
        lab = obj['simpleLabelFromStatusByFinalRejection']
    except:
        return None
    return lab
    
def map_func(obj):
    claim = get_claim(obj)
    label = get_label(obj)
    return claim, label

    

map_func(comprehensive.find_one())

(['protective',
  'composite',
  'fabric',
  'comprising',
  'least',
  'one',
  'layer',
  'base',
  'fabric',
  'wherein',
  'base',
  'fabric',
  'densely',
  'woven',
  'ballistic',
  'fabric',
  'b',
  'least',
  'one',
  'rigid',
  'ballistic',
  'fabric',
  'layer',
  'base',
  'layer',
  'hard',
  'polymer',
  'layer',
  'special',
  'matrix',
  'bonded',
  'onto',
  'base',
  'layer',
  'least',
  'one',
  'rigid',
  'ballistic',
  'fabric',
  'layer',
  'cut',
  'strips',
  'material',
  'scale-like',
  'pattern',
  'shape',
  'connected',
  'least',
  'one',
  'layer',
  'base',
  'fabric',
  'overlapping',
  'layers',
  'whereby',
  'least',
  'one',
  'layer',
  'base',
  'fabric',
  'covered',
  'c',
  'resilient',
  'elastically',
  'deformable',
  'material',
  'least',
  'one',
  'sensor',
  'therein',
  'monitoring',
  'vital',
  'signs',
  'wearer',
  'resilient',
  'elastically',
  'deformable',
  'material',
  'association',
  'least',
  'one',
  'layer',
  'base',

In [59]:
# get_claim(comprehensive.find_one())
comprehensive.estimated_document_count()

6299342

In [60]:
query = {"simpleLabelFromStatusByFinalRejection": { "$type" : "int" }, 
         "publications": { "$type" : "array" }}


In [196]:
filtered_comp = comprehensive.find(query)

# dataset_size = 9999
# break_pt = 0
with open('cleaned_data.txt', 'a') as file:
    for i in filtered_comp:
    #     print(i)
        k, l = map_func(i)
        if (k != None) and (l != None):
            key = ','.join(k)
            to_write = key + ',' + str(l) + '\n'
            file.write(to_write)
#     if break_pt == dataset_size:
#         break
#     break_pt += 1


KeyboardInterrupt: 

# 3. Building Model (borrow from XW)

In [208]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


In [279]:
def build_corpus(fp = 'cleaned_data.txt'):
    out_corpus = []
    out_labels = []
    with open(fp, 'r') as file:
        for i in file:
            feat, lab = i[:-2], i[-2]
            out_corpus.append(feat)
            out_labels.append(lab)
    return out_corpus, out_labels

In [280]:
corpus,labels = build_corpus()
print(len(corpus))
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, labels, test_size = 0.2, random_state = 0)

39407


In [325]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(corpus_train)
X_test = vectorizer.transform(corpus_test)

clf = LogisticRegression(max_iter = 1500, C = 2).fit(X_train, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [326]:
clf.score(X_train,y_train)

0.7840444091990484

In [327]:
clf.score(X_test, y_test)

0.6787617356001014

In [328]:
tn, fp, fn, tp = confusion_matrix(y_test, clf.predict(X_test)).ravel()

precision = tp / (tp + fp)
recall = tp / (tp + fn)
TNR = tn / (tn + fp)
NPV = tn / (tn + fn)

In [329]:
precision,recall, TNR, NPV

(0.7131817506315946,
 0.8885391594149231,
 0.22208786779524387,
 0.4778837814397225)

In [330]:
np.array([tn, fp, fn, tp]) / 7882

array([0.06990612, 0.24486171, 0.07637655, 0.60885562])

In [331]:
np.mean(np.array(y_test) == '1')

0.685232174574981

In [332]:
np.mean(np.array(y_test) == '0')

0.314767825425019

In [333]:
pred_y = clf.predict(X_test)

In [334]:
np.mean(np.array(pred_y) == '1')

0.8537173306267445

In [335]:
np.mean(np.array(pred_y) == '0')

0.14628266937325551